# Note: Move this to the home directory of your TPU if you want to load both LingvoTF and MaxText config

## If creating new checkpoint disable eager mode, if continuing training from the checkpoint written out, enable eager mode, and directly go to MaxText part

In [1]:
import lingvo.compat as tf
# tf.disable_eager_execution()


tf.enable_eager_execution()


2023-10-14 08:55:11.769156: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-14 08:55:11.769193: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
import importlib
#the Lingvot TF experiment
module = importlib.import_module('lg-gpt3.lingvo.tasks.lm.params.lg_gpt3')

2023-10-14 08:50:12.609981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-14 08:50:12.610022: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
module

<module 'lg-gpt3.lingvo.tasks.lm.params.lg_gpt3' from '/home/mazumdera/lg-gpt3/lingvo/tasks/lm/params/lg_gpt3.py'>

In [4]:

sess = tf.Session()

tf_model = module.LgTestModel().Task()
tf_model = tf_model.Instantiate()

2023-10-14 08:50:27.295618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-14 08:50:27.295650: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-14 08:50:27.295680: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t1v-n-8e231585-w-0): /proc/driver/nvidia/version does not exist
2023-10-14 08:50:27.296194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
tf_model

In [6]:
# download the checkpoint if not already done
#!gsutil -m cp -r  "gs://mazumdera-test-bucket/lingvo/v4-16-LgTestModel-10102023-1/train/*" ~/lg-lingvo-checkpoint/


In [7]:
import os
from pprint import pprint

tf_path = os.path.abspath(
    './lg-lingvo-checkpoint')  # Path to our TensorFlow checkpoint

#load the Lingvo TF checkpoint
reader=tf.train.load_checkpoint(tf_path)

In [8]:
ckpt_shape_map = reader.get_variable_to_shape_map()
ckpt_shape_map

{'transformer/decoder/final_layer_norm/w/scale/var/Adafactor': [512],
 'transformer/decoder/final_layer_norm/w/scale/var': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var/Adafactor': [384],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var': [384,
  512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wk/var/Adafactor_1': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wk/var/Adafactor': [384],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var/Adafactor_1': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_001/dense_relu_dense/w/wo/var': [2048,
  512],
 'transformer/dec_pos_emb/w/embedding/var/Adafactor': [512],
 'transformer/decoder/blocks/iter_00000/blocks_body/layer_000/ln/w/scale/var/Adafactor': [512],
 'transformer/dec_emb/w/embedding/var/Adafactor_1': [50272],
 

## MaxText part

In [18]:
#Get all the imports
import jax
import os
import sys

jax.config.update('jax_default_prng_impl', 'unsafe_rbg')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
os.environ["LIBTPU_INIT_ARGS"] = os.environ.get("LIBTPU_INIT_ARGS","") + " --xla_tpu_spmd_rng_bit_generator_unsafe=true"
print(f"Found {jax.device_count()} devices.")

from typing import Sequence
import datetime
from absl import app
from flax.linen import partitioning as nn_partitioning
from flax import linen as nn
import numpy as np
import optax
from tensorboardX import SummaryWriter

from layers import Transformer
import pyconfig
from input_pipeline import create_data_iterator_with_tokenizer
import max_utils
import checkpointing

import jax.numpy as jnp
from jax import random
from jax.sharding import PartitionSpec as P
from jax.sharding import Mesh

from jax.experimental.compilation_cache import compilation_cache as cc

from cloud_tpu_diagnostics import diagnostic
from cloud_tpu_diagnostics.configuration import debug_configuration
from cloud_tpu_diagnostics.configuration import diagnostic_configuration
from cloud_tpu_diagnostics.configuration import stack_trace_configuration

import max_utils
from jax.sharding import PartitionSpec as P
import max_logging
from tensorboardX import SummaryWriter

Found 8 devices.


In [3]:
from flax.training import train_state

In [4]:
base_output_directory="base_output_directory=gs://mazumdera-test-bucket/maxtext/lg/10142023/1"
file_pattern_for_train_data="file_pattern_for_train_data=gs://yejingxin-us-central2/external/lg/dummy-data/train/*.tfrecords"
file_pattern_for_eval_data="file_pattern_for_eval_data=gs://yejingxin-us-central2/external/lg/dummy-data/valid/*tfrecords"
base_num_decoder_layers="base_num_decoder_layers=1"
base_num_heads = "base_num_heads=4"
head_nums = "head_dim=96"
dataset_type = "dataset_type=lg"

In [5]:
commandline_args = ["dummy", "/home/mazumdera/maxtext/MaxText/configs/base.yml","run_name=1xv3-8", "dcn_data_parallelism=1", "save_period=5","ici_data_parallelism=4","ici_tensor_parallelism=2","steps=20","enable_profiler=true","remat_policy=full","base_emb_dim=512", base_num_heads, head_nums,"vocab_size=50272", base_num_decoder_layers, "per_device_batch_size=0.5","enable_profiler=true", "base_mlp_dim=2048", file_pattern_for_train_data, file_pattern_for_eval_data, base_output_directory, dataset_type,"max_predict_length=512"]


In [6]:
pyconfig.initialize(commandline_args)
config = pyconfig.config

In [7]:
checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(
      config.checkpoint_dir,
      config.enable_checkpointing,
      config.async_checkpointing,
      config.save_period,
  )

Creating checkpoint manager...


2023-10-14 08:55:43.536689: I external/xla/xla/pjrt/distributed/service.cc:492] Experimental coordination service is enabled.
2023-10-14 08:55:43.536971: I external/xla/xla/pjrt/distributed/service.cc:524] Jax service listening on 10.128.0.69:43329
2023-10-14 08:55:43.537720: I external/tsl/tsl/distributed_runtime/coordination/coordination_service.cc:551] /job:jax_worker/replica:0/task:0 has connected to coordination service. Incarnation: 8997740297104136765
2023-10-14 08:55:43.537868: I external/tsl/tsl/distributed_runtime/coordination/coordination_service_agent.cc:299] Coordination agent has successfully connected.
2023-10-14 08:55:43.538114: I external/xla/xla/pjrt/distributed/client.cc:508] Connected to distributed JAX controller


Checkpoint manager created!


In [8]:
# Initial PRNG Keys
init_rng, nextrng = random.split(random.PRNGKey(config.init_weights_seed), 2)

In [9]:
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)] (num_devices: 8)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1)]]

 [[TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1)]]

 [[TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0)
   TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]]

 [[TpuDevic

In [10]:
model = Transformer(config, mesh)

In [11]:
#TODO: also compare with optax.adafactor
tx = optax.adamw(
      max_utils.create_learning_rate_schedule(config),
      b1=config.adam_b1,
      b2=config.adam_b2,
      eps=config.adam_eps,
      eps_root=config.adam_eps_root,
      weight_decay=config.adam_weight_decay,
  )

## Run the following cells if creating the tranlated ckpt, else skip to running the new ckpt

In [19]:
state, state_mesh_annotations = max_utils.setup_initial_state(model, tx, config, init_rng, mesh, checkpoint_manager)


No existing checkpoints found, not restoring checkpoint.


In [20]:
import functools
import max_utils
init_train_state_partial = functools.partial(max_utils.init_train_state, model, tx,
                                               config)

In [21]:
#get the flax.training.train_state.TrainState
abstract_state = jax.eval_shape(init_train_state_partial, init_rng)
abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fee7a3a7c10>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 1, 2048), dtype=float32), names=('embed', 'layers', 'mlp'), mesh=None, rules=None)}, 'wo': {'kernel': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(2048, 1, 512), dtype=float32), names=('mlp', 'layers', 'embed'), mesh=None, rules=None)}}, 'pre_self_attention_layer_norm': {'scale': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(512, 1), dtype=float32), names=('embed', 'layers'), mesh=None, rules=None)}, 'relpos_bias': {'rel_embedding': LogicallyPartitioned(value=ShapeDtypeStruct(shape=(4, 1, 32), dtype=float32), nam

In [22]:

state_logical_annotations = nn.get_partition_spec(abstract_state)
state_logical_annotations

TrainState(step=PartitionSpec(), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fee7a3a7c10>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': PartitionSpec('embed', 'layers', 'mlp')}, 'wo': {'kernel': PartitionSpec('mlp', 'layers', 'embed')}}, 'pre_self_attention_layer_norm': {'scale': PartitionSpec('embed', 'layers')}, 'relpos_bias': {'rel_embedding': PartitionSpec('heads', 'layers', 'relpos_buckets')}, 'self_attention': {'key': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'key_layer_norm': {'scale': PartitionSpec('heads', 'layers')}, 'out': {'kernel': PartitionSpec('heads', 'layers', 'kv', 'embed')}, 'query': {'kernel': PartitionSpec('embed', 'layers', 'heads', 'kv')}, 'query_layer_norm': {'scale': PartitionSpec('heads', 'l

In [23]:
unboxed_abstract_state = max_utils.unbox_logicallypartioned_trainstate(abstract_state)
unboxed_abstract_state

TrainState(step=ShapeDtypeStruct(shape=(), dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fee7a3a7c10>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': ShapeDtypeStruct(shape=(512, 1, 2048), dtype=float32)}, 'wo': {'kernel': ShapeDtypeStruct(shape=(2048, 1, 512), dtype=float32)}}, 'pre_self_attention_layer_norm': {'scale': ShapeDtypeStruct(shape=(512, 1), dtype=float32)}, 'relpos_bias': {'rel_embedding': ShapeDtypeStruct(shape=(4, 1, 32), dtype=float32)}, 'self_attention': {'key': {'kernel': ShapeDtypeStruct(shape=(512, 1, 4, 96), dtype=float32)}, 'key_layer_norm': {'scale': ShapeDtypeStruct(shape=(96, 1), dtype=float32)}, 'out': {'kernel': ShapeDtypeStruct(shape=(4, 1, 96, 512), dtype=float32)}, 'query': {'kernel': ShapeDtypeS

In [24]:
#Num of params in the MaxText config

from train import calculate_num_params_from_pytree
print(f"Total num of params = {calculate_num_params_from_pytree(state.params)}")

Found 8 devices.
Total num of params = 28624192


In [25]:
# #Let us consider we are saving a state at step 0 at the vanilla state
# import max_logging
# step = 0
# if checkpoint_manager is not None:
#       if checkpoint_manager.save(step, state):
#         max_logging.log(f"saved a checkpoint at step {step}")

## Create a copy of the state and modify this one, and compare with the previous copy

In [26]:
state_new, state_mesh_annotations_new = max_utils.setup_initial_state(model, tx, config, init_rng, mesh, checkpoint_manager)


No existing checkpoints found, not restoring checkpoint.


## Update the MaxText state with values from the LingvoTF Checkpoint

In [27]:
with sess.as_default():
    
    #mlp mapping
    out_tensor = reader.get_tensor('transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wo/var')
    print(f"out_tensor_shape = {out_tensor.shape}")
    out_tensor_reshaped = sess.run(tf.reshape(out_tensor, [config.base_num_heads, config.base_num_decoder_layers, config.head_dim,config.base_emb_dim]))
    print(f"out_tensor_reshaped.shape = {out_tensor_reshaped.shape}")
    print(f"out_tensor.shape = {out_tensor.shape}")
    print(f"out_tensor_reshaped = {out_tensor_reshaped}")
    state_new.params['decoder']['decoder']['self_attention']['out']['kernel'] = out_tensor_reshaped

    
    wo_tensor = reader.get_tensor('transformer/decoder/blocks/iter_00000/blocks_body/layer_001/dense_relu_dense/w/wo/var')
    print(f"wo shape = {wo_tensor.shape}")
    wo_tensor_reshaped = sess.run(tf.reshape(wo_tensor, [config.base_mlp_dim,config.base_num_decoder_layers, config.base_emb_dim]))
    print(f"wo_tensor_reshaped.shape = {wo_tensor_reshaped.shape}")
    print(f"wo_tensor_reshaped = {wo_tensor_reshaped}")
    state_new.params['decoder']['decoder']['mlp']['wo']['kernel'] = wo_tensor_reshaped

    wk_tensor = reader.get_tensor('transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wk/var')
    print(f"wk_tensor shape = {wk_tensor.shape}")
    wk_tensor_reshaped = sess.run(tf.reshape(wk_tensor, [config.base_emb_dim,config.base_num_decoder_layers, config.base_num_heads, config.head_dim]))
    print(f"wk_tensor_reshaped.shape = {wk_tensor_reshaped.shape}")
    print(f"wk_tensor_reshaped = {wk_tensor_reshaped}")                              
    state_new.params['decoder']['decoder']['self_attention']['key']['kernel'] = wk_tensor_reshaped


    wq_tensor = reader.get_tensor('transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wq/var')
    print(f"wq_tensor shape = {wq_tensor.shape}")
    wq_tensor_reshaped = sess.run(tf.reshape(wq_tensor, [config.base_emb_dim,config.base_num_decoder_layers, config.base_num_heads, config.head_dim]))
    print(f"wq_tensor_reshaped.shape = {wq_tensor_reshaped.shape}")
    print(f"wq_tensor_reshaped = {wq_tensor_reshaped}")
    state_new.params['decoder']['decoder']['self_attention']['query']['kernel'] = wq_tensor_reshaped

    wv_tensor = reader.get_tensor('transformer/decoder/blocks/iter_00000/blocks_body/layer_000/dec_self_attention/w/wv/var')
    print(f"wv_tensor shape = {wv_tensor.shape}")
    wv_tensor_reshaped = sess.run(tf.reshape(wv_tensor, [config.base_emb_dim,config.base_num_decoder_layers, config.base_num_heads, config.head_dim]))
    print(f"wv_tensor_reshaped.shape = {wv_tensor_reshaped.shape}")
    print(f"wv_tensor_reshaped = {wv_tensor_reshaped}")
    state_new.params['decoder']['decoder']['self_attention']['value']['kernel'] = wv_tensor_reshaped

    
    wi_tensor = reader.get_tensor('transformer/decoder/blocks/iter_00000/blocks_body/layer_001/dense_relu_dense/w/wi/var')
    print(f"wi_tensor shape = {wi_tensor.shape}")
    wi_tensor_reshaped = sess.run(tf.reshape(wi_tensor, [config.base_emb_dim,config.base_num_decoder_layers, config.base_mlp_dim]))
    print(f"wi_tensor_reshaped.shape = {wi_tensor_reshaped.shape}")
    print(f"wi_tensor_reshaped = {wi_tensor_reshaped}")
    state_new.params['decoder']['decoder']['mlp']['wi']['kernel'] = wi_tensor_reshaped


    #input embedding
    embedding_tensor = reader.get_tensor('transformer/dec_emb/w/embedding/var')
    print(f"embedding_tensor shape = {embedding_tensor.shape}")
    print(f"embedding_tensor.shape = {embedding_tensor.shape}")
    state_new.params['token_embedder']['embedding'] = embedding_tensor




out_tensor_shape = (384, 512)
out_tensor_reshaped.shape = (4, 1, 96, 512)
out_tensor.shape = (384, 512)
out_tensor_reshaped = [[[[-1.53098423e-02 -6.04504123e-02 -4.67418768e-02 ...  1.24808691e-01
    -7.74728283e-02  1.05696999e-01]
   [-2.23743562e-02  6.17731847e-02  2.63397992e-02 ...  7.06597120e-02
    -1.47394575e-02 -3.44053097e-02]
   [-1.35133499e-02  1.27908960e-02 -3.51429805e-02 ... -2.11554449e-02
     2.99066640e-02 -4.58324775e-02]
   ...
   [-1.40595183e-01 -3.62042822e-02 -4.52025309e-02 ... -2.03563389e-03
     8.36239606e-02 -8.50903988e-02]
   [ 3.52970790e-03  4.84331371e-03  3.11585944e-02 ...  1.71957035e-02
     6.75476268e-02  2.61918530e-02]
   [-3.52053791e-02 -2.48367134e-02  5.24204113e-02 ... -2.92290971e-02
     8.25344473e-02 -4.25014794e-02]]]


 [[[-5.46128042e-02  1.73444524e-02  7.59419724e-02 ... -1.37610603e-02
     7.93458894e-02 -8.05024151e-03]
   [-3.50178481e-04 -1.90739534e-04 -7.16518611e-02 ... -1.78389810e-02
     7.25608096e-02  7.12766

2023-10-14 08:51:27.256952: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


wi_tensor_reshaped.shape = (512, 1, 2048)
wi_tensor_reshaped = [[[-0.03641239 -0.03962705  0.05871062 ...  0.07531501 -0.07563277
    0.01552885]]

 [[ 0.0077048  -0.03596908  0.06365027 ...  0.04490145  0.03413538
   -0.06831469]]

 [[-0.01209099 -0.05410476  0.04568633 ...  0.07060074 -0.02464857
    0.00637501]]

 ...

 [[ 0.07429314 -0.02990046 -0.05796176 ... -0.05946061 -0.03993283
   -0.01406334]]

 [[ 0.02112295  0.00023006  0.06490169 ...  0.06899361 -0.04490572
   -0.01001916]]

 [[-0.01463708  0.05100304  0.03406622 ... -0.06231261 -0.0555151
    0.06860438]]]
embedding_tensor shape = (50272, 512)
embedding_tensor.shape = (50272, 512)


In [49]:
# state_new.params['decoder']['decoder']['mlp']['wi']['kernel'].shape == state.params['decoder']['decoder']['mlp']['wi']['kernel'].shape

True

## Write out the new state

In [51]:
# config.checkpoint_dir
# !gsutil ls gs://mazumdera-test-bucket/maxtext/lg/10142023/1/1xv3-8/checkpoints/

gs://mazumdera-test-bucket/maxtext/lg/10142023/1/1xv3-8/checkpoints/


In [28]:
# #since we know know we are loading checkpoint at step 50
step = 50
if checkpoint_manager is not None:
      if checkpoint_manager.save(step, state_new):
        max_logging.log(f"saved a checkpoint at step {step}")

I0000 00:00:1697273634.292325  437021 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1697273634.294070  438809 google_auth_provider.cc:179] Running on GCE, using service account 630405687483-compute@developer.gserviceaccount.com


saved a checkpoint at step 50


## Now try training with the newly saved model state but need to restart kernel and start with TF eager mode

In [22]:

state_read_from_ckpt, state_mesh_annotations_read_from_ckpt = max_utils.setup_initial_state(model, tx, config, init_rng, mesh, checkpoint_manager)


restoring state from this run's directory latest step         50


In [23]:
state_read_from_ckpt

TrainState(step=Array(0, dtype=int32), apply_fn=<bound method Module.apply of Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7fcbe3d56620>
    mesh = Mesh(device_ids=array([[[0, 1]],
    
           [[2, 3]],
    
           [[6, 7]],
    
           [[4, 5]]]), axis_names=('data', 'fsdp', 'tensor'))
)>, params={'decoder': {'decoder': {'mlp': {'wi': {'kernel': Array([[[-0.03641239, -0.03962705,  0.05871062, ...,  0.07531501,
         -0.07563277,  0.01552885]],

       [[ 0.0077048 , -0.03596908,  0.06365027, ...,  0.04490145,
          0.03413538, -0.06831469]],

       [[-0.01209099, -0.05410476,  0.04568633, ...,  0.07060074,
         -0.02464857,  0.00637501]],

       ...,

       [[ 0.07429314, -0.02990046, -0.05796176, ..., -0.05946061,
         -0.03993283, -0.01406334]],

       [[ 0.02112295,  0.00023006,  0.06490169, ...,  0.06899361,
         -0.04490572, -0.01001916]],

       [[-0.01463708,  0.05100304,  0.03406622, ..., -0.06231261,
    

In [24]:
from train import calculate_num_params_from_pytree
import max_logging
import train

writer = SummaryWriter(config.tensorboard_dir)

learning_rate_schedule = max_utils.create_learning_rate_schedule(config)
data_iterator = create_data_iterator_with_tokenizer(config, mesh)

data_pspec = P(*config.data_sharding)


num_model_parameters = calculate_num_params_from_pytree(state_read_from_ckpt.params)
max_logging.log(f"number parameters: {num_model_parameters/10**9:.3f} billion")
per_device_tflops = train.calculate_training_tflops(num_model_parameters, config)

# Define compiled top-level functions.
state_mesh_shardings_read_from_ckpt = jax.tree_map(
  lambda p: jax.sharding.NamedSharding(mesh, p), state_mesh_annotations_read_from_ckpt)
data_sharding = jax.tree_map(
  lambda p: jax.sharding.NamedSharding(mesh, p), data_pspec)
p_train_step = jax.jit(
train.train_step,
in_shardings=(state_mesh_shardings_read_from_ckpt, data_sharding, None),
  out_shardings=(state_mesh_shardings_read_from_ckpt, None, None),
static_argnums=(0,1,),
donate_argnums=2)

example_batch = None
last_step_completion = datetime.datetime.now()

local_metrics_file = open(config.metrics_file, 'a', encoding="utf8") if config.metrics_file else None
running_gcs_metrics = [] if config.gcs_metrics else None

#ckpt was for step=50, let us train 40 more steps
step = 90
for step in np.arange(train.get_first_step(state_read_from_ckpt), step):
    example_batch = train.load_next_batch(data_iterator, example_batch, config)
    with mesh, nn_partitioning.axis_rules(config.logical_axis_rules):
      state_read_from_ckpt, metrics, nextrng = p_train_step(
          model, config, state_read_from_ckpt, example_batch, nextrng
      )
    
    new_time = datetime.datetime.now()
    train.record_scalar_metrics(metrics, new_time - last_step_completion,  per_device_tflops, learning_rate_schedule(step))
    train.write_metrics(writer, metrics, step, config)
    last_step_completion = new_time
    
    if checkpoint_manager is not None:
      if checkpoint_manager.save(step, state_read_from_ckpt):
        max_logging.log(f"saved a checkpoint at step {step}")
      # Upon preemption, exit when and only when all ongoing saves are complete.
      if checkpoint_manager.reached_preemption(step):
        checkpoint_manager.wait_until_finished()
        sys.exit()
    
    if config.metrics_file:
      max_utils.write_metrics_locally(metrics, step, config, local_metrics_file)
    
    if config.gcs_metrics and jax.process_index() == 0:
      running_gcs_metrics = max_utils.write_metrics_for_gcs(metrics, step, config, running_gcs_metrics)
    
    # Start profiling at end of first step to avoid compilation.
    # Move before for loop to include.
    if step == 0:
      max_utils.activate_profiler(config)

max_utils.deactivate_profiler(config)
writer.close()

Trying to read training data from path: gs://yejingxin-us-central2/external/lg/dummy-data/train/*.tfrecords
Training dataset has: -2 entries
Trying to read eval data from path: gs://yejingxin-us-central2/external/lg/dummy-data/valid/*tfrecords
Eval dataset has: -2 entries
number parameters: 0.029 billion
Per train step, total TFLOPs will be 0.19, split as 94.79% learnable weight flops and 5.21% attention flops
completed step: 0, seconds: 2.360, TFLOP/s: 0.079, loss: 15.419
To see full metrics 'tensorboard --logdir=gs://mazumdera-test-bucket/maxtext/lg/10142023/1/1xv3-8/tensorboard/'
completed step: 1, seconds: 0.892, TFLOP/s: 0.208, loss: 15.397
completed step: 2, seconds: 0.037, TFLOP/s: 5.058, loss: 15.382
completed step: 3, seconds: 0.037, TFLOP/s: 4.989, loss: 15.308
completed step: 4, seconds: 0.037, TFLOP/s: 5.061, loss: 15.243


2023-10-14 09:05:40.765354: I external/tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-10-14 09:05:40.765401: I external/tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


completed step: 5, seconds: 0.037, TFLOP/s: 5.038, loss: 15.171
completed step: 6, seconds: 0.037, TFLOP/s: 4.961, loss: 15.123
completed step: 7, seconds: 0.038, TFLOP/s: 4.884, loss: 15.025
completed step: 8, seconds: 0.037, TFLOP/s: 4.953, loss: 15.006
completed step: 9, seconds: 0.038, TFLOP/s: 4.890, loss: 14.934
completed step: 10, seconds: 0.037, TFLOP/s: 5.036, loss: 14.861
completed step: 11, seconds: 0.039, TFLOP/s: 4.764, loss: 14.820
completed step: 12, seconds: 0.037, TFLOP/s: 5.068, loss: 14.739
completed step: 13, seconds: 0.035, TFLOP/s: 5.246, loss: 14.701
completed step: 14, seconds: 0.034, TFLOP/s: 5.435, loss: 14.659
completed step: 15, seconds: 0.037, TFLOP/s: 5.051, loss: 14.671
completed step: 16, seconds: 0.037, TFLOP/s: 5.061, loss: 14.665
completed step: 17, seconds: 0.036, TFLOP/s: 5.151, loss: 14.626
completed step: 18, seconds: 0.034, TFLOP/s: 5.418, loss: 14.617
completed step: 19, seconds: 0.033, TFLOP/s: 5.554, loss: 14.614
completed step: 20, seconds: 0

2023-10-14 09:11:52.437660: I external/tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2023-10-14 09:12:15.850645: I external/tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: gs://mazumdera-test-bucket/maxtext/lg/10142023/1/1xv3-8/tensorboard/plugins/profile/2023_10_14_09_12_13/t1v-n-8e231585-w-0.xplane.pb
2023-10-14 09:12:24.575929: I external/tsl/tsl/profiler/rpc/client/save_profile.cc:117] Creating directory: gs://mazumdera-test-bucket/maxtext/lg/10142023/1/1xv3-8/tensorboard/plugins/profile/2023_10_14_09_12_13

2023-10-14 09:12:30.069546: I external/tsl/tsl/profiler/rpc/client/save_profile.cc:123] Dumped gzipped tool data for trace.json.gz to gs://mazumdera-test-bucket/maxtext/lg/10142023/1/1xv3-8/tensorboard/plugins/profile/2023_10_14_09_12_13/t1v-n-8e231585-w-0.trace.json.gz
2023-10-14 09:12:30.518623: I external/tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
